## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure (api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Nizwa,22.9333,57.5333,77.72,26,0,6.04,OM,clear sky
1,1,Poum,-20.2333,164.0167,81.99,82,100,17.81,NC,moderate rain
2,2,Provideniya,64.3833,-173.3000,2.80,74,0,11.50,RU,clear sky
3,3,Mar Del Plata,-38.0023,-57.5575,75.22,64,40,21.85,AR,scattered clouds
4,4,Beloha,-25.1667,45.0500,71.80,88,1,10.36,MG,clear sky


In [3]:
city_data_df.dtypes

City_ID          int64
City            object
Lat            float64
Lng            float64
Max Temp       float64
Humidity         int64
Cloudiness       int64
Wind Speed     float64
Country         object
Description     object
dtype: object

In [4]:
max_temp = city_data_df["Max Temp"]
temps=[]
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & \
                                       (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Nizwa,22.9333,57.5333,77.72,26,0,6.04,OM,clear sky
1,1,Poum,-20.2333,164.0167,81.99,82,100,17.81,NC,moderate rain
3,3,Mar Del Plata,-38.0023,-57.5575,75.22,64,40,21.85,AR,scattered clouds
4,4,Beloha,-25.1667,45.0500,71.80,88,1,10.36,MG,clear sky
5,5,Bambous Virieux,-20.3428,57.7575,80.22,83,75,11.50,MU,broken clouds
6,6,Vaini,-21.2000,-175.2000,73.56,100,100,5.75,TO,light intensity shower rain
11,11,Victoria,22.2855,114.1577,65.25,61,0,11.99,HK,clear sky
12,12,Butaritari,3.0707,172.7902,81.46,72,98,14.50,KI,overcast clouds
13,13,Hilo,19.7297,-155.0900,76.24,93,32,1.01,US,moderate rain
15,15,Puerto Suarez,-18.9500,-57.8000,89.13,46,100,5.95,BO,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        369
City           369
Lat            369
Lng            369
Max Temp       369
Humidity       369
Cloudiness     369
Wind Speed     369
Country        364
Description    369
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID        364
City           364
Lat            364
Lng            364
Max Temp       364
Humidity       364
Cloudiness     364
Wind Speed     364
Country        364
Description    364
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Nizwa,OM,77.72,clear sky,22.9333,57.5333,
1,Poum,NC,81.99,moderate rain,-20.2333,164.0167,
3,Mar Del Plata,AR,75.22,scattered clouds,-38.0023,-57.5575,
4,Beloha,MG,71.80,clear sky,-25.1667,45.0500,
5,Bambous Virieux,MU,80.22,broken clouds,-20.3428,57.7575,
6,Vaini,TO,73.56,light intensity shower rain,-21.2000,-175.2000,
11,Victoria,HK,65.25,clear sky,22.2855,114.1577,
12,Butaritari,KI,81.46,overcast clouds,3.0707,172.7902,
13,Hilo,US,76.24,moderate rain,19.7297,-155.0900,
15,Puerto Suarez,BO,89.13,overcast clouds,-18.9500,-57.8000,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000, "type": "lodging", "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.count()

City           342
Country        342
Max Temp       342
Description    342
Lat            342
Lng            342
Hotel Name     342
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))